In [3]:
# Install required packages
!pip install -q syncode transformers torch

from syncode import Syncode
import re

# Define the grammar for extracting operands and operator
MATH_GRAMMAR = """
start: number "," number "," op

number: /-?[0-9]+\\.?[0-9]*/
op: "+" | "-" | "*" | "/"

%import common.WS
%ignore WS
"""

def run_syncode(textual_query: str) -> tuple:
    """
    Extracts operands and operator from a mathematical query using SynCode,
    then computes and returns the result.

    Args:
        textual_query: Natural language math query (e.g., "What is 327 multiplied by 11.0?")

    Returns:
        tuple: (ordered list of operands, operator as string, final answer as float)
    """

    # Create concise few-shot examples
    few_shot_examples = """Extract operands and operator:

Query: What is 327 multiplied by 11.0?
327.0,11.0,*

Query: What is 45.1 plus 23.54?
45.1,23.54,+

Query: What is 120.4 divided by 4.0?
120.4,4.0,/

Query: What is 50 minus 20?
50.0,20.0,-

"""

    # Initialize SynCode with Phi-2 model
    syncode = Syncode(
        model="microsoft/phi-2",
        mode="grammar_strict",
        grammar=MATH_GRAMMAR,
        parse_output_only=True
    )

    # Create the full prompt - keep it concise
    prompt = few_shot_examples + f"Query: {textual_query}\n"

    print(f"Processing: {textual_query}")

    # Generate output using SynCode
    output = syncode.infer(prompt)[0]

    print(f"Raw output: '{output}'")

    # Parse the output - handle both with and without spaces
    output_clean = output.strip().replace(' ', '')
    parts = output_clean.split(',')

    if len(parts) != 3:
        raise ValueError(f"Unexpected output format: '{output}'. Got {len(parts)} parts instead of 3.")

    operand1 = float(parts[0])
    operand2 = float(parts[1])
    operator = parts[2]

    # Compute the result using Python
    if operator == '+':
        result = operand1 + operand2
    elif operator == '-':
        result = operand1 - operand2
    elif operator == '*':
        result = operand1 * operand2
    elif operator == '/':
        result = operand1 / operand2
    else:
        raise ValueError(f"Unknown operator: {operator}")

    return ([operand1, operand2], operator, result)


# Test the function with the provided examples
if __name__ == "__main__":
    test_queries = [
        "What is 327 multiplied by 11.0?",
        "What is 45.1 plus 23.54?",
        "What is 120.4 divided by 4.0?"
    ]

    print("Starting tests...\n")

    for i, query in enumerate(test_queries, 1):
        print(f"Test {i}/{len(test_queries)}")
        try:
            operands, operator, answer = run_syncode(query)
            print(f"✓ Operands: {operands}")
            print(f"✓ Operator: {operator}")
            print(f"✓ Final Answer: {answer}")
        except Exception as e:
            print(f"✗ Error: {e}")
        print("-" * 60)

Starting tests...

Test 1/3
[2025-12-12 07:50:58,010-root] - Loading model microsoft/phi-2 with device:cuda, device_map:auto, torch_dtype:torch.bfloat16


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Processing: What is 327 multiplied by 11.0?
Raw output: '327.0,11.0,*


'
✓ Operands: [327.0, 11.0]
✓ Operator: *
✓ Final Answer: 3597.0
------------------------------------------------------------
Test 2/3
[2025-12-12 07:52:36,319-root] - Loading model microsoft/phi-2 with device:cuda, device_map:auto, torch_dtype:torch.bfloat16


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Processing: What is 45.1 plus 23.54?
Raw output: '45.1,23.54,+



'
✓ Operands: [45.1, 23.54]
✓ Operator: +
✓ Final Answer: 68.64
------------------------------------------------------------
Test 3/3
[2025-12-12 07:54:09,861-root] - Loading model microsoft/phi-2 with device:cuda, device_map:auto, torch_dtype:torch.bfloat16


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Processing: What is 120.4 divided by 4.0?
Raw output: '120.4,4.0,/



'
✓ Operands: [120.4, 4.0]
✓ Operator: /
✓ Final Answer: 30.1
------------------------------------------------------------
